In [1]:
import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.float_format='{:.2f}'.format
plt.style.use('seaborn')
import sklearn as skl
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
#for the tuner
import tensorflow as tf
import keras_tuner as kt
#
import io
import shutil
import os
import warnings
warnings.filterwarnings("ignore")

C:\Users\rober\AppData\Local\Temp\ipykernel_10668\4255545000.py:4: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
fpth='mushroom_data_clean.csv'
df=pd.read_csv(fpth)
df.head()
# df.describe()
# df.info()
# for c in df.columns:
#     print(c,df[c].unique())

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above,stalk-color-below,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e


In [3]:
df.drop(columns=['stalk-root'],inplace=True)
df.head()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above,stalk-color-below,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e


In [4]:
df['poisonous']=df['class']
df['poisonous']=df['poisonous'].apply(lambda x:1 if x in 'p' else 0)
y=df['poisonous']
df.drop(columns=['class','poisonous'],inplace=True)
X=df

In [5]:
X=pd.get_dummies(X)
X

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
8120,0,0,0,0,0,1,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
8121,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
8122,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,1,0,0,0,0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,stratify=y)

In [7]:
try:
    dirpath=('./untitled_project')
    shutil.rmtree(dirpath)
    os.mkdir(dirpath)
except: pass

In [8]:
#tuner parameters
EPOCHS=5
ITERATIONS=5
MAX_LAYERS=10
MAX_NODES=20

In [9]:
# Create a method that creates a new Sequential model with hyperparameter options
INPUT_DIM=X_train.shape[1]
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    activation = hp.Choice('activation',['relu','tanh','sigmoid','softmax'])
    hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam'])
    hp_lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    if hp_optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=hp_lr)
    elif hp_optimizer == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp_lr)
    elif hp_optimizer == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=hp_lr)
    else:
        raise
    
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=MAX_NODES,
        step=2), activation=activation, input_dim=INPUT_DIM))
    for i in range(hp.Int('num_layers', 1, MAX_LAYERS)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=MAX_NODES,
            step=2),
            activation=activation))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return nn_model

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=EPOCHS,
    hyperband_iterations=ITERATIONS)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(6093, 112) (6093,) (2031, 112) (2031,)


In [10]:
tuner.search(X_train,y_train,epochs=EPOCHS,validation_data=(X_test,y_test))

Trial 50 Complete [00h 00m 08s]
val_accuracy: 0.8606597781181335

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 03m 36s
INFO:tensorflow:Oracle triggered exit


In [21]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

64/64 - 0s - loss: 0.0019 - accuracy: 1.0000 - 344ms/epoch - 5ms/step
Loss: 0.001871682470664382, Accuracy: 1.0


In [22]:
best_hyper.values

{'activation': 'tanh',
 'optimizer': 'rmsprop',
 'learning_rate': 0.001,
 'first_units': 13,
 'num_layers': 8,
 'units_0': 13,
 'units_1': 1,
 'units_2': 13,
 'units_3': 13,
 'units_4': 1,
 'units_5': 7,
 'units_6': 3,
 'units_7': 13,
 'units_8': 13,
 'units_9': 13,
 'tuner/epochs': 5,
 'tuner/initial_epoch': 2,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0003'}